# "You Snooze, You Win" Challenge

Every year, the [PhysioNet/CinC (Computing in Cardiology) Challenge](https://www.physionet.org/challenge/) invites "participants to tackle clinically interesting problems that are either unsolved or not well-solved." As you may have figured out from the title, this year's challenge focuses on sleep, particularly the classification of nonarousal and arousal timeframes. If you would like to understand the biological implications of the challenge, we recommend reading PhysioNet's [introduction](https://physionet.org/challenge/2018/) of the challenge.

For this challenge, you will classify samples into 5 classes (Arousal, NREM1, NREM2, NREM3, REM). Each sample consists of seven physiological signals (O2-M1, E1-M2, Chin1-Chin2, ABD, CHEST, AIRFLOW, ECG) measured at 200 Hz over a 60 second period (12000 timepoints). In this notebook, we provide code to import the data, visualize sample signals, implement an example classifier, and 'score' your model.

In [1]:
### Run once ###
! pip install peakutils
from scipy.interpolate import *
import peakutils
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sys
import random
from sklearn import metrics
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from scipy.signal import *
! pip install neurokit
import neurokit as nk
tf.enable_eager_execution()

## Loading the Dataset

This dataset is a modified version of the PhysioNet/CinC Challenge data, which were contributed by the Massachusetts General Hospital’s Computational Clinical Neurophysiology Laboratory, and the Clinical Data Animation Laboratory.
***
**Class labels:**
- 0 = Arousal
- 1 = NREM1
- 2 = NREM2
- 3 = NREM3
- 4 = REM
***
**Class description:**

|Class|State|Characterizations|Brain Waves|Percentage of Sleep|
|-|-|-|-|-|-|
|Arousal|Consciousness|Wakefulness (coherent cognitive and behavioral responses to external stimuli)|Alpha, Beta|-|
|NREM1|Light sleep|Hypnic jerk (involuntary twitch that causes an individual to awaken for a moment)|Theta|5|
|NREM2|Unequivocal sleep|Sleep spindle (sudden burst of oscillatory brain activity); K-complex (delta wave that lasts for a second)|Theta, Delta|40-50|
|NREM3|Deep sleep|Parasomnias (sleep disorders such as sleepwalking and night terrors)|Delta|15-25|
|REM|Dreaming sleep|REM atonia (paralysis of nonessential skeletal muscles); Dreaming|Alpha, Beta|20-25|
***
**Physiological signal description:**

O2-M1 - posterior brain activity (electroencephalography)

E1-M2 - left eye activity (electrooculography)

Chin1-Chin2 - chin movement (electromyography)

ABD - abdominal movement (electromyography)

CHEST - chest movement (electromyography)

AIRFLOW - respiratory airflow

ECG - cardiac activity (electrocardiography)
***
Run both cell blocks to get the challenge data.

In [2]:
! git clone https://github.com/BeaverWorksMedlytics/datasets.git

fatal: destination path 'datasets' already exists and is not an empty directory.


In [0]:
os.chdir('./datasets/Week2_Challenge/')

## Data Visualization

Run the first cell once to store the training and test sample file locations. If the data have been imported correctly, the cell should output '4000' (training) and '1000' (test).

Run the second cell once to initialize important functions that you may find useful. Descriptions of input and output will be provided for every function in the notebook.

Run the last cell to visualize a random test sample's seven physiological signals in a raw and FFT (fast fourier transform) format. Note that you can change different parameters, which we will go over in detail.

It is important to recognize that the same signal from different samples in the same class may vary in terms of amplitude and frequency. Of course, this is a byproduct of intraspecies variation. Further data preprocessing and/or discriminatory feature extraction may be necessary to account for this phenomenon.

In [4]:
### Run once ###

def get_file_locs():
    file_dict = {'training':[], 'test':[]}
    for data_type in file_dict:
        for file in os.listdir('./' + data_type):
            file_dict[data_type].append(data_type + '/' + file)
    """ file_dict (dict) stores 'training' and 'test' as keys and sample file locations as values """
    return file_dict

file_dict = get_file_locs()
print(len(file_dict['training']), len(file_dict['test']))

file_dict = get_file_locs()
print(len(file_dict['training']), len(file_dict['test']))

4000 1000
4000 1000


In [0]:
def get_sample_data(data_type, id_number):
    file = './' + data_type + '/' + str(id_number) + '.xz'
    sample = pd.read_pickle('./' + file)
    return sample, file.split('/')[2]

def get_raw_signals(sample):
    raw_signals_x = np.arange(0, 60, step = 1/200)
    raw_signals_y = np.transpose(sample['Signal'][0])
    """ tuple: raw_signals_x (ndarray) stores raw signal timepoints in seconds; raw_signals_y (ndarray) stores all seven signals for every timepoint; 'Raw' (str) stores signal type """
    return (raw_signals_x, raw_signals_y, 'Raw')

def get_data(data_type, num_samples): # transposed into 4000, 7 
    raw, order = np.array([]), np.array([])
    for i in range(num_samples):
        sample, sample_id = get_sample_data(data_type, i)
        print("dt",data_type, "i",i, "sample_id",sample_id)
        print("sample:", sample)
        print()
        sample_raw = get_raw_signals(sample)
        np.append(raw, sample_raw)
        print(sample_raw)
        print(raw)
        sys.exit(0)
    return raw, order
 

In [17]:
train_data = get_data('training', 4000)
test_data = get_data('test', 4000)

### Run once ###

train_labels = np.ndarray(shape = (1, 4000))
for i in range(4000):
    train_labels[0][i] = i//800
train_labels = tf.keras.utils.to_categorical(train_labels[0], 5)
train_labels = train_labels.astype(np.float32)

print(train_labels)

### Run every time you modify your feature extraction ###
train_data = train_data.astype(np.float32)
test_data = test_data.astype(np.float32)
train_labels = train_labels.astype(np.float32)

print(train_data.shape, train_labels.shape, '\n\n', train_data, '\n\n', train_labels)

dt training i 0 sample_id 0.xz
sample:                                               Signal  Label
0  [[3.0, -9.0, -4.0, 71.0, -9.0, 19.0, 1.0], [0....      0

(array([0.0000e+00, 5.0000e-03, 1.0000e-02, ..., 5.9985e+01, 5.9990e+01,
       5.9995e+01]), array([[   3.,    0.,    1., ...,   -6.,   -1.,    4.],
       [  -9.,  -11.,  -11., ...,    5.,    4.,   -4.],
       [  -4.,   -7.,    4., ...,   24.,   10.,    3.],
       ...,
       [  -9.,   -9.,  -10., ..., -113., -113., -113.],
       [  19.,   20.,   19., ...,   25.,   26.,   26.],
       [   1.,  -10.,   -6., ...,  -10.,   -9.,  -20.]]), 'Raw')
[]


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
### Run every time you modify your feature extraction ###

train_data, train_labels = shuffle(train_data, train_labels)

""" val_size (int) must be from 0-4000 """
val_size = 1000
mocktest_size = 500

val_data = train_data[:val_size]
mocktest_data = train_data[val_size:val_size + mocktest_size]
partial_train_data = train_data[val_size + mocktest_size:]

val_labels = train_labels[:val_size]
mocktest_labels = train_labels[val_size:val_size + mocktest_size]
partial_train_labels = train_labels[val_size + mocktest_size:]

training_set = tf.data.Dataset.from_tensor_slices((partial_train_data, partial_train_labels))
training_set = training_set.shuffle(partial_train_labels.shape[0]).batch(40)

twod_array = np.ndarray(shape=(4000, 84000))
for i in range(4000):
  sample, sample_id = get_sample_data('training', i)
  sample_raw = get_raw_signals(sample)
  pancake = sample_raw[1].flatten()
  twod_array[i] = pancake
  
print(twod_array)

## Example Classifier

Below is an example of a poor classifier for this dataset. It is a simple neural network that takes the maximal, normalized variance for each signal (given a frame and interval) as its input features. After training, it performs with 44% accuracy on test data.

While the example classifier makes use of a neural network, we encourage you to utilize any ML algorithm that you feel would be appropriate. Also, note that this example classifier makes use of only the raw signals, without consideration of the FFT signals (or any other processed form of the raw signals).

In [0]:
### Run every time you change your model parameters ###

#model = tf.keras.Sequential()

""" Modify to your heart's and algorithm's content ^_^ """

#model.add(keras.layers.Conv1D(64, kernel_size=3,
#                 activation=tf.nn.relu,
#                 input_shape=(4000,7)))

#model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(train_data.shape[1],)))
#model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
#model.add(tf.keras.layers.Dense(5, activation=tf.nn.softmax))
#model.compile(loss='categorical_crossentropy', optimizer=tf.train.RMSPropOptimizer(learning_rate=0.001), metrics=['accuracy'])

#model.summary()

model = keras.Sequential()
model.add(keras.layers.Conv1D(filters=18, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=twod_array[0].shape))
#model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=1, padding='same', activation='relu')

model.add(keras.layers.MaxPooling1D(pool_size= 2))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())

# model.add(Dropout(0.5))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(twod_array, train_labels, 
          batch_size=100,
          epochs=20,
          verbose=1)
          #validation_data=(x_test, y_test))

In [0]:
### Run every time you change your model parameters ###

#model = tf.keras.Sequential()

""" Modify to your heart's and algorithm's content ^_^ """

#model.add(keras.layers.Conv1D(64, kernel_size=3,
#                 activation=tf.nn.relu,
#                 input_shape=(4000,7)))

#model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(train_data.shape[1],)))
#model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
#model.add(tf.keras.layers.Dense(5, activation=tf.nn.softmax))
#model.compile(loss='categorical_crossentropy', optimizer=tf.train.RMSPropOptimizer(learning_rate=0.001), metrics=['accuracy'])

#model.summary()

model = keras.Sequential()
model.add(keras.layers.Conv1D(filters=18, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=twod_array[0].shape))
#model.add(keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=1, padding='same', activation='relu')

model.add(keras.layers.MaxPooling1D(pool_size= 2))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())

# model.add(Dropout(0.5))
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(twod_array, train_labels, 
          batch_size=100,
          epochs=20,
          verbose=1)
          #validation_data=(x_test, y_test))

In [0]:
### Run whenever you want to train and validate your model ###

""" EPOCHS (int) is whatever number causes stabilization of validation loss and accuracy """
EPOCHS = 50

for epoch in range(EPOCHS):
    for signals, labels in training_set:
        tr_loss, tr_accuracy = model.train_on_batch(signals, labels)
    val_loss, val_accuracy = model.evaluate(val_data, val_labels)
    print(('Epoch #%d\t Training Loss: %.2f\tTraining Accuracy: %.2f\t'
         'Validation Loss: %.2f\tValidation Accuracy: %.2f')
         % (epoch + 1, tr_loss, tr_accuracy,
         val_loss, val_accuracy))

## Submitting Your Model

After training your classifier, run it on the test data to generate your predictions. Each class for a test sample should have an associated probability (between 0 and 1). Below are the parameters for the prediction format and export:

- Your predictions should be in a pandas dataframe with 5 columns (classes) and 1000 rows (samples). Note that your predictions must follow the original test sample order (0.xz, 1.xz, 2.xz, ...). You only need to worry about this if you shuffled the test samples or stored the samples in an unordered data structure (dictionaries and sets). If this is the case, you should 1) add a separate column in your pandas dataframe with the file number for each sample; 2) sort the dataframe using this column; and 3) drop the column. These steps have been noted in the code below.
- The predictions dataframe should be exported as an .xz file using dataframe.to_pickle() followed by files.download().

Example code of the prediction format and export is presented in the cell block below. 

In [0]:
### Run once after you have finished training your model ###

test_pred = model.predict(test_data)
test_output = np.ndarray(shape = (1000, 6))

""" Add column with file number """
for i in range(1000):
    test_output[i] = np.append(test_pred[i], test_order[i])
test_dataframe = pd.DataFrame(test_output)

""" Sort dataframe according to file number """
sorted_test_dataframe = test_dataframe.sort_values(by=[5])

""" Drop file number column """
processed_test_dataframe = sorted_test_dataframe.drop(sorted_test_dataframe.columns[5], axis=1)

print(test_dataframe.head(), '\n\n', sorted_test_dataframe.head(), '\n\n', processed_test_dataframe.head())

file = 'PotatoesAreGreat.xz'
processed_test_dataframe.to_pickle(file)
files.download(file)

Your model will be evaluated on Area Under the ROC Curve (ROCAUC), Matthews Correlation Coefficient (MCC) and creativity. There will be a "winning" group for each of these categories.

If you are finished early, consider trying other ML algorithms and/or implementing multiple feature extraction methods. You can also help other groups if you finish early.

## How Your Model Will Be Evaluated

- **Area Under the ROC Curve (AUCROC)**: The receiver operating characteristic (ROC) curve plots the true positive rate (sensitivity/recall) against the false positive rate (fall-out) at many decision threshold settings. The area under the curve (AUC) measures discrimination, the classifier's ability to correctly identify samples from the "positive" and "negative" cases. Intuitively, AUC is the probability that a randomly chosen "positive" sample will be labeled as "more positive" than a randomly chosen "negative" sample. In the case of a multi-class ROC curve, each class is considered separately before taking the weighted average of all the class results. Simply put, the class under consideration is labeled as "positive" while all other classes are labeled as "negative." Below is the multi-class ROC curve for the example classifier. The AUCROC score should be between 0 and 1, in which 0.5 is random classification and 1 is perfect classification.

 ![alt text](https://image.ibb.co/g4pzST/ROCAUC.png)

- **Matthews Correlation Coefficient (MCC)**: The MCC measures the quality of binary classifications, irrespective of the class sizes. Importantly, it is typically regarded as a balanced measure since it considers all values in the 2x2 contingency table (TP, FP, TN, FN). For this challenge, the binary classes will be "Arousal" (Arousal) and "Nonarousal" (NREM1, NREM2, NREM3, REM). The MCC score should be between -1 and 1, in which 0 is random classification and 1 is perfect classification.

 ![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/5caa90fc15105b74b59a30bbc9cc2e5bd43a13b7)

Using these metrics, the example classifier has the following scores on test data:
- AUCROC: 0.755
- MCC: 0.353
- Creativity: ( ͡° ͜ʖ ͡°)

Below is the code used to calculate the AUCROC and MCC metrics when evaluating your classifier.

In [0]:
test_pred = pd.DataFrame(model.predict(mocktest_data))
test_predict = test_pred.idxmax(axis=1)
test_labels = [ np.where(label==1)[0][0] for label in mocktest_labels]
test_labels_one_hot = pd.DataFrame(mocktest_labels)

In [0]:
fpr = {}
tpr = {}
roc_auc = {}
for i in range(5):
    fpr[i], tpr[i], _ = metrics.roc_curve(test_labels_one_hot.iloc[:, i], test_pred.iloc[:, i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], label = stage_dict[i] + ', ' + str(i))
plt.plot([0, 1], [0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-Class ROC Curve')
plt.legend()
plt.show()

fpr["micro"], tpr["micro"], _ = metrics.roc_curve(test_labels_one_hot.values.ravel(), test_pred.values.ravel())
roc_auc = metrics.auc(fpr["micro"], tpr["micro"])

In [0]:
y_true = []
y_pred = []
for i in range(test_pred.shape[0]):
    if test_predict.iloc[i]==0: y_pred.append(1)
    else: y_pred.append(-1)
    if test_labels[i]==0: y_true.append(1)
    else: y_true.append(-1)
mcc = metrics.matthews_corrcoef(y_true, y_pred)

In [0]:
print(roc_auc, mcc)